In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Dropout, BatchNormalization, GRU
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import mixed_precision
from IPython.display import display, Image

# ---------- Enable Mixed Precision Training ----------
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# ---------- Load and clean data ----------
df = pd.read_csv("yhuo0768_A2_CleanDataset(2).csv")
df = df[df['Sex'].isin(['Male', 'Female'])].copy()
df['Sex'] = df['Sex'].map({'Male': 0, 'Female': 1})

# ---------- Feature Engineering ----------
# Z-score 标准化构建 HealthRiskScore
for col in ['Physical (days)', 'BMI']:
    df[col + '_z'] = (df[col] - df[col].mean()) / df[col].std()
df['HealthRiskScore'] = df['BMI_z'] * 0.6 + df['Physical (days)_z'] * 0.4

# 添加二次项与交互项
df['BMI_squared'] = df['BMI'] ** 2
df['Age_BMI'] = df['Age'] * df['BMI']

# ---------- Optimized LSTM Pipeline with KFold Cross Validation ----------
def build_optimized_lstm(X_train, y_train, X_val, y_val, lstm_units=64, epochs=100, batch_size=64):
    # 构建优化的LSTM模型
    model = Sequential()
    model.add(Bidirectional(LSTM(lstm_units, return_sequences=True), input_shape=(X_train.shape[1], 1)))
    model.add(LSTM(lstm_units, return_sequences=False))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))  # 减少dropout
    model.add(Dense(lstm_units // 2, activation='relu'))
    model.add(Dropout(0.3))  # 减少dropout
    model.add(Dense(y_train.shape[1], activation='softmax'))  # 分类任务

    # 使用 Adam 优化器并设置学习率调度
    optimizer = Adam(learning_rate=0.001)
    
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    # 设置回调
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3)

    # 训练模型
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
                        epochs=epochs, batch_size=batch_size, 
                        callbacks=[early_stop, lr_scheduler], verbose=2)

    return model, history

def run_lstm_pipeline_with_kfold(df_input, features, target, lstm_units=64, epochs=100, batch_size=64, name_prefix=""):

    df_model = df_input[features + [target]].dropna().copy()
    X = df_model[features].values
    y = df_model[target].values

    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    num_classes = len(le.classes_)
    y_cat = to_categorical(y_encoded, num_classes=num_classes)

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    timesteps = X_scaled.shape[1]
    X_seq = X_scaled.reshape((X_scaled.shape[0], timesteps, 1))

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    fold = 1
    avg_acc = []

    for train_index, val_index in kf.split(X_seq):
        print(f"\nTraining fold {fold}...")
        X_train, X_val = X_seq[train_index], X_seq[val_index]
        y_train, y_val = y_cat[train_index], y_cat[val_index]

        smote = SMOTE(random_state=42)
        X_train_flat = X_train.reshape((X_train.shape[0], timesteps))
        y_train_flat = np.argmax(y_train, axis=1)
        X_res_flat, y_res_flat = smote.fit_resample(X_train_flat, y_train_flat)
        X_res = X_res_flat.reshape((X_res_flat.shape[0], timesteps, 1))
        y_res = to_categorical(y_res_flat, num_classes=num_classes)

        # 调用优化后的LSTM模型
        model, history = build_optimized_lstm(X_res, y_res, X_val, y_val, lstm_units, epochs, batch_size)

        # Evaluate
        y_pred_prob = model.predict(X_val)
        y_pred = np.argmax(y_pred_prob, axis=1)
        y_true = np.argmax(y_val, axis=1)

        acc = accuracy_score(y_true, y_pred)
        avg_acc.append(acc)
        print(f"Fold {fold} accuracy: {acc:.4f}")
        fold += 1

    print(f"\nAverage accuracy across all folds: {np.mean(avg_acc):.4f}")

    # Save final model and scaler
    model.save(f"{name_prefix}{target}_lstm_model.h5")
    joblib.dump(scaler, f"{name_prefix}{target}_scaler.pkl")

    return model, scaler

# ---------- Stage 1 ----------
features_1 = ['Age', 'Sex', 'Physical (days)', 'BMI', 'HealthRiskScore', 'BMI_squared', 'Age_BMI']
target_1 = 'GeneralHealth'
model_gh, scaler_gh = run_lstm_pipeline_with_kfold(
    df, features_1, target_1,
    lstm_units=64, epochs=100, batch_size=64,  # 调整批次大小和LSTM单元数
    name_prefix="gh_"
)

# ---------- Stage 2 ----------
clear_df2 = df.dropna(subset=features_1 + ['Diabetes']).copy()
X2_features = clear_df2[features_1].values
X2_scaled = scaler_gh.transform(X2_features)
X2_seq = X2_scaled.reshape((X2_scaled.shape[0], X2_scaled.shape[1], 1))

pred_probs_gh = model_gh.predict(X2_seq)
clear_df2['PredictedGeneralHealth'] = np.argmax(pred_probs_gh, axis=1)

features_2 = features_1 + ['PredictedGeneralHealth']
target_2 = 'Diabetes'
model_db, scaler_db = run_lstm_pipeline_with_kfold(
    clear_df2, features_2, target_2,
    lstm_units=64, epochs=100, batch_size=64,  # 调整批次大小和LSTM单元数
    name_prefix="db_"
)

print("✅ All models trained and saved with enhanced LSTM pipeline.")
